In [1]:
#Устанавливаем pySpark
!pip install pyspark >> None

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, date_format, first, round

In [19]:
# Создаем SparkSession
spark = SparkSession.builder.appName("AverageRevenue").getOrCreate()

# Создаем DataFrame из предоставленных данных
data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])

# Определяем окно для расчета среднего значения
window = Window.partitionBy("category").orderBy("category")

# Вычисляем среднюю выручку для каждой категории
df_avg = df.withColumn("avg_revenue", round(avg("revenue").over(window), 1))
df_avg.show()


+----------+-----------+--------+-------+-----------+
|      date|   category|quantity|revenue|avg_revenue|
+----------+-----------+--------+-------+-----------+
|2023-11-20|      Books|     150|   9000|    10333.3|
|2023-11-21|      Books|     200|  12000|    10333.3|
|2023-11-22|      Books|     180|  10000|    10333.3|
|2023-11-20|   Clothing|     300|  15000|    15000.0|
|2023-11-21|   Clothing|     280|  14000|    15000.0|
|2023-11-22|   Clothing|     320|  16000|    15000.0|
|2023-11-20|Electronics|     100|  12000|    12500.0|
|2023-11-21|Electronics|     110|  13000|    12500.0|
|2023-11-22|Electronics|     105|  12500|    12500.0|
+----------+-----------+--------+-------+-----------+



In [20]:
# Применяем операцию pivot, чтобы преобразовать данные
df_pivot = df_avg.groupBy("category").pivot("date").agg(first("avg_revenue"))


# Выводим результат
df_pivot.show()

# Останавливаем SparkSession
spark.stop()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|   10333.3|   10333.3|   10333.3|
|   Clothing|   15000.0|   15000.0|   15000.0|
|Electronics|   12500.0|   12500.0|   12500.0|
+-----------+----------+----------+----------+

